In [1]:
#Procedimiento realizado a la tabla precios_semanas_20200419_20200426.xlsx
#  especificamente precios_20200419
#   1.Se borran los datos repetidos si existen y los valores null

import pandas as pd
import xlrd

df_ps_20200419 = pd.read_excel('Datasets\precios_semanas_20200419_20200426.xlsx ','precios_20200419_20200419')

#-------------Tratamiento General
# Se borran valores repetidos
df_ps_20200419.drop_duplicates(inplace=True,ignore_index=True)

#Se borran valores nulos
df_ps_20200419 = df_ps_20200419.dropna()
#df_ps_20200419.info()
#df_ps_20200419.sample(5)


#-----------Manejo de los Id del columna sucursal_id
#Se convierte el codigo string en una lista con 
# los subcodigos (comercioId-banderaId-sucursalId.) 
Ids =[]
for id_Tex in df_ps_20200419["sucursal_id"]:
    try:
        Ids.append(id_Tex.split("-")) 
    except:
        Ids.append(None)
        
#Se procede a crear la nueva columna con los nuevos Id_Sucursal
Id_Sucursal =[]
for Idcompuesta in Ids:
    if Idcompuesta:
        num = int(Idcompuesta[0])*10000*100  +  int(Idcompuesta[1])*10000 + int(Idcompuesta[2])
        Id_Sucursal.append(num)
    else:
        Id_Sucursal.append(None)

#Se agrega la columna Id_Sucursal con valores de id int
df_ps_20200419.insert(loc = 0 , column = "Id_Sucursal", value = Id_Sucursal)

#Se guardan valores con problemas, con mas tiempo se pueden mejorar e integrar a la tabla
df_ps_20200419_erroresId = df_ps_20200419[df_ps_20200419["Id_Sucursal"].isnull()].drop(["Id_Sucursal"], axis=1)

#Se borran valores nulos o con problemas
df_ps_20200419.dropna(subset="Id_Sucursal",inplace=True)

#Se elimina la columna sucursal_id
df_ps_20200419.drop(["sucursal_id"], axis=1,inplace=True)

df_ps_20200419["Id_Sucursal"] = df_ps_20200419["Id_Sucursal"].astype('int')


#------------Manejo de los Id del columna producto_id
#Se verifica el producto_id creando lista para la columna con valores Id 
# que se puedan convertir en int 
df_ps_20200419["producto_id"] = df_ps_20200419["producto_id"].astype('string').str.rstrip('.0')

id_producto =[]
for i in range(len(df_ps_20200419)):
    try:
        id_producto.append(int(df_ps_20200419.iloc[i,2]))
    except ValueError:
        id_producto.append(None)
        
#Se agrega la columna ID_producto con valores de id int
df_ps_20200419.insert(loc = 0 , column = "Id_producto", value = id_producto)

#Se guardan valores con problemas, con mas tiempo se pueden mejorar e integrar a la tabla
df_ps_20200419_erroresId = df_ps_20200419[df_ps_20200419["Id_producto"].isnull()].drop(["Id_producto"], axis=1)

#Se borran valores nulos o con problemas
df_ps_20200419.dropna(subset="Id_producto",inplace=True)

#Se elimina la columna producto_id
df_ps_20200419.drop(["producto_id"], axis=1,inplace=True)

#Se exporta la base de datos a MySQL
import sqlalchemy as db
database_username='root'            # Nombre de cliente en MySQL Workbrench
database_password='JCSR.MySQL123'   # Contraseña de MySQL Workbrench
database_ip='localhost'
database_name='ETL_Lab01'           # Nombre de Base de datos a donde nos conectaremos (se debio crear primero en Mysql)
database_conection = db.create_engine(f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}',
                                    encoding='latin1')
coneccion=database_conection.connect()
df_ps_20200419.to_sql('ps_20200419',coneccion, index=False)

#Creacion de conexion con Mysql 
import pymysql
conexion = pymysql.connect(
   host = 'localhost',
   user = 'root',
   passwd='JCSR.MySQL123',
   db='ETL_Lab01'           #LA base de datos se debio crear primero en Mysql
)
cursor = conexion.cursor()
try:
    #Se crean las relaciones
    cursor.execute("SET FOREIGN_KEY_CHECKS=0;")
    #cursor.execute("alter table ps_20200419 add foreign key (Id_Sucursal) references sucursal(Id_Sucursal);")
    cursor.execute("alter table ps_20200419 add foreign key (Id_producto) references producto(Id_producto);")
    cursor.execute("SET FOREIGN_KEY_CHECKS=1;")
finally:
    conexion.close()
print("Carga de tabla 'ps_20200419' a MySQL completa")

Carga de tabla 'ps_20200419' a MySQL completa
